# Train the model for further analysis using Grad_Cam
Here, models are output without a nested layer structure.

The difference between this file and file "main_TL_vs_Scracth":

(1) In file "utils.train_val_test_dataset_import_GC", dataset are rescaled and then put into the model without rescaling layer.

(2) In file "models.TL_models_GC", model are built without rescaling layer and the pre-trained model (e.g., mobilenetv2_1.00_224) are not nested as a layer named "mobilenetv2_1.00_224"

In [ ]:
from utils.miscellaneous import read_config
from copy import deepcopy


import utils.train_val_test_dataset_import_GC as tvt_GC
import utils.class_imbalances as ci
import utils.plots as plot
import models.TL_models_GC as TL
import tensorflow as tf
import matplotlib.pyplot as plt


## Parse configuration file + initializations

In [ ]:
# read config files
cfg = read_config('./config.yaml')

# constants
image_height = cfg['image_height']
image_width = cfg['image_width']
batch_size = cfg['batch_size']['tra']
# num_epochs = cfg['trainParams']['num_epochs']
num_epochs = 100
lr_rate = cfg['adamParams']['lr']
num_classes = cfg['num_classes']


# paths
path_train = r"F:\Tianlong\Data\Deep_Plastic\Data\Exp4_binary\Train 1"
path_val = r"F:\Tianlong\Data\Deep_Plastic\Data\Exp4_binary\Validation 1"

# load datasets
ds_train, ds_val = tvt_GC.import_dataset_train_val(
    path_train, path_val, image_height, image_width, batch_size)

# autotune
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
ds_val = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

# class weights
class_weights_train = ci.class_weights_binary(path_train)

# paths to model and checkpoint file save
save_model_path_TL_all = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp4_binary\densenet121_TL_all_lr_0.0001_binary.hdf5"
save_ckp_path_TL_all = r"F:\Tianlong\Data\Deep_Plastic\Results\Exp4_binary\trained_checkpoint"

In [ ]:
num_classes

In [ ]:
path_train

In [ ]:
path_val

## Train models

### Transfer learning, fine-tune all layers
pre-trained on ImageNet and fine-tune all layers

In [ ]:
# train models


lr_rate=0.0001
r_dense_a = TL.DenseN121(num_classes, 'TL_all', class_weights=class_weights_train, save_model_path=save_model_path_TL_all, 
    save_ckp_path=save_ckp_path_TL_all, 
    image_height=image_height, image_width=image_width, 
    ds_train=ds_train, ds_val=ds_val, lr_rate=lr_rate, num_epochs=num_epochs)
val_acc = r_dense_a.history['val_accuracy']
print("Best Validation Accuracy is", max(val_acc))

### Plot accuracy and loss

In [ ]:
# Plot model accuracy and loss


# plot.plot_hist(hist=r_mobileV2, model_name="MobileNetV2")

# plot.plot_hist(hist=r_squeeze, model_name="SqueezeNet")

# plot.plot_hist(hist=r_resnet_s, model_name="ResNet50")
# plot.plot_hist(hist=r_resnet_c, model_name="ResNet50")
# plot.plot_hist(hist=r_resnet_a, model_name="ResNet50")

# plot.plot_hist(hist=r_inception_s, model_name='InceptionV3')
# plot.plot_hist(hist=r_inceptionr_c, model_name='InceptionV3')
# plot.plot_hist(hist=r_inception_a, model_name='InceptionV3')

#plot.plot_hist(hist=r_dense_s, model_name="DenseNet121")
#plot.plot_hist(hist=r_dense_c, model_name="DenseNet121")
plot.plot_hist(hist=r_dense_a, model_name="DenseNet121")

### Tensorboard

In [ ]:
# load the tensorboard

%load_ext tensorboard

# if the tensorboard page on VS Code is not so clear, 
# you can type this (localhost:6006) on web browser after executing this code 